In [1]:
import torch
from pykeen.evaluation import evaluate, RankBasedEvaluator
from pykeen.metrics.ranking import HitsAtK
import pandas as pd


import logging
from pathlib import Path

import click
import more_click
import torch
from pykeen.evaluation import RankBasedEvaluator
from pykeen.losses import NSSALoss,CrossEntropyLoss
from pykeen.models.inductive import InductiveNodePiece, InductiveNodePieceGNN
from pykeen.trackers import ConsoleResultTracker, WANDBResultTracker, FileResultTracker
from pykeen.training import SLCWATrainingLoop
from pykeen.typing import TESTING, TRAINING, VALIDATION
from pykeen.utils import resolve_device, set_random_seed
from torch.optim import Adam


from pykeen.metrics.ranking import HitsAtK

from pathlib import Path

from pykeen.datasets.inductive.base import DisjointInductivePathDataset
from pykeen.datasets.base import PathDataset, Dataset
from typing_extensions import Literal
import os
from pykeen.hpo import hpo_pipeline
from pykeen.triples import TriplesFactory
from pykeen.models import InductiveNodePiece, TransE, RGCN, ConvE
from pykeen.typing import TESTING, TRAINING, VALIDATION

import time

import platform
import sys

import cpuinfo

import psutil

import subprocess

import zipfile

from tqdm import tqdm

from IPython.display import clear_output

seed = 1234

/proj/naiss2023-22-172/users/x_giaru/anaconda3/envs/thesis/lib/python3.10/site-packages/pykeen/evaluation/evaluator.py:16: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


In [2]:

# specify the path to the zip file and the destination directory for the unzipped files
zip_file_path = "Testing Traces/MSCallGraph_traces.zip"
extract_dir = "Testing Traces/MSCallGraph_traces"

# create a ZipFile object and extract all files to the destination directory
with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(extract_dir)


In [3]:
def show_metrics(dictionary,model_name,csv_name):
    for key in dictionary.keys():
#         print(key)
        df = pd.DataFrame(dictionary[key])
        df.to_csv(f"{model_name}_testing_traces/{csv_name}_{key}.csv")
#         print(df)
        return df

In [4]:
DATA_TYPE = "_transductive.tsv"
TRAIN_PATH = "MSCallGraph_train" + DATA_TYPE
TEST_PATH = "MSCallGraph_test" + DATA_TYPE
VALIDATE_PATH = "MSCallGraph_validation" + DATA_TYPE

In [5]:
dataset = PathDataset(training_path = TRAIN_PATH,
                     testing_path = TEST_PATH,
                     validation_path = VALIDATE_PATH,
                      eager = True,
                     create_inverse_triples = True
                     )


In [9]:
def test_on_traces(model_name):
    directory = model_name+"_testing_traces"

    if not os.path.exists(directory):
        os.makedirs(directory)
        print(f'Directory {directory} created successfully!')
    else:
        print(f'Directory {directory} already exists.')
        
    # specify the path to the zip file and the destination directory for the unzipped files
    zip_file_path = model_name+".zip"
    extract_dir = ""

    # create a ZipFile object and extract all files to the destination directory
    with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
        zip_ref.extractall(extract_dir)

    metrics = ['meanreciprocalrank', HitsAtK(1),
                 HitsAtK(3), HitsAtK(5), HitsAtK(10)]

    train_evaluator = RankBasedEvaluator(
            metrics=metrics,
            add_defaults=False,
        )
    valid_evaluator = RankBasedEvaluator(
            metrics=metrics,
            add_defaults=False,
        )
    test_evaluator = RankBasedEvaluator(
            metrics = metrics,
            add_defaults=False
        )
    
    model = torch.load(f"{model_name}/model.pth")
    
    def test_trace(traceid):
#     TRACE_DATA_TYPE = "_trace_0b133c1915919238193454000e5d37.tsv"

        TRACE_TRAIN_PATH = "Testing Traces/MSCallGraph_traces/Testing Traces/MSCallGraph_traces/train/" + traceid + "_transductive_train.tsv"
        TRACE_TEST_PATH = "Testing Traces/MSCallGraph_traces/Testing Traces/MSCallGraph_traces/test/" +traceid + "_transductive_test.tsv"
        # TRACE_VALIDATE_PATH = "MSCallGraph_0_validation" + TRACE_DATA_TYPE

        trace_dataset = PathDataset(training_path = TRACE_TRAIN_PATH,
                             testing_path = TRACE_TEST_PATH,
                             validation_path = TRACE_TEST_PATH,

                              eager = True
                             )
        trace_testing = TriplesFactory.from_path(
        TRACE_TEST_PATH,
        entity_to_id=dataset.training.entity_to_id,
        relation_to_id=dataset.training.relation_to_id,
    )
        trace_training = TriplesFactory.from_path(
            TRACE_TRAIN_PATH,
            entity_to_id=dataset.training.entity_to_id,
            relation_to_id=dataset.training.relation_to_id,
        )


        # result on the test set
        print("Trace Test error")
        return show_metrics(test_evaluator.evaluate(
                model=model,
                mapped_triples=trace_testing.mapped_triples,

                additional_filter_triples=[
                    # filtering of other positive triples
                trace_testing.mapped_triples,
                trace_training.mapped_triples
                ],
                restrict_entities_to = dataset.training.entities_to_ids(list(trace_dataset.training.entity_to_id.keys())),
                restrict_relations_to = dataset.training.relations_to_ids(list(trace_dataset.training.relation_to_id.keys())),
            ).to_dict(),model_name,f'{traceid}_test_metrics')

    

    # Specifica il percorso della cartella da cui si vogliono ottenere i nomi dei file
    folder_path = "Testing Traces/MSCallGraph_traces/Testing Traces/MSCallGraph_traces/train/"

    files_list = []
    for filename in os.listdir(folder_path):
        # Ottenere il nome del file
        file_name = os.path.basename(filename)
        files_list.append(file_name)
    all_traces_list = []
    # Scansione di ogni file nella cartella
    with tqdm(desc=f'{model_name} testing traces', total=len(files_list)) as progress_bar:
        for file_name in files_list:
            # Stampa il nome del file
            all_traces_list.append(test_trace(file_name[:-23]))
            progress_bar.update(1)
            clear_output(wait=True)


    mean_trace_df = sum(all_traces_list)/len(all_traces_list)
    mean_trace_df.to_csv(f"{model_name}_testing_traces/mean_trace_test_metrics.csv")
    
    def zip_folder(folder_path, output_path):
        with zipfile.ZipFile(output_path, 'w', zipfile.ZIP_DEFLATED) as zipf:
            for root, dirs, files in os.walk(folder_path):
                for file in files:
                    zipf.write(os.path.join(root, file))

    folder_path = model_name+"_testing_traces"
    output_path = f'{folder_path}.zip'

    zip_folder(folder_path, output_path)

In [10]:
model_name_list = ['convE_transductive','tuckER_transductive','rotatE_transductive','rgcn_transductive','quatE_transductive',
                   'nodepiece_transductive']

In [11]:
for model_name in model_name_list:
    test_on_traces(model_name)
    clear_output(wait=True)

